# Obraz - transfer learning - praca domowa
Ostatnia aktualizacja: 2022.12.10

Z dokumentacji [Keras](https://keras.io/guides/transfer_learning/): *Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.*

Głównym celem pracy domowej jest stworzenie klasyfikatora, który będzie odróżniał zdjęcia obiektów w wybranym przez nas zbiorze. Użyjemy do tego jednego z [gotowych modeli Keras](https://keras.io/api/applications/), wytrenowanego wcześniej na zbiorze Imagenet. 

Przydatne źródła:
- [transfer learning vgg16 + tf_flowers](https://towardsdatascience.com/transfer-learning-with-vgg16-and-keras-50ea161580b4)
- [Keras - transfer learning](https://keras.io/guides/transfer_learning/)

### Wybierz swój zbiór danych### 
Chcemy mieć co najmniej 2-3 klasy. Możesz użyć gotowego zbioru (np. z [katalogu tensorflow](https://www.tensorflow.org/datasets/catalog/overview) lub [kaggle](https://www.kaggle.com/datasets)) albo użyć własnych danych. 
- [Przykładowy zbiór: tf_flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers). 
- [Przykładowy zbiór: plant_leaves](https://www.tensorflow.org/datasets/catalog/plant_leaves). 

Wczytaj i przygotuj dane do treningu. 



In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical

(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"],
    batch_size=-1,
    as_supervised=True,
)

train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [2]:
print(train_ds[1][1][149])

tf.Tensor([0. 0. 0.], shape=(3,), dtype=float32)


### Wczytaj wytrenowany model do klasyfikacji obrazu. ###
Może to być jeden z gotowych [modeli dostępnych w Keras](https://keras.io/api/applications/). Wczytujemy go z wytrenowanymi już wcześniej wagami na Imagenecie (weights='imagenet'). 

Model możemy wczytać bez ostatnich warstw (include_top=False) i dodać je potem ręcznie, dostosowane do liczby klas w naszym zbiorze. Imagenet ma 1000 klas, my prawdopodobnie będziemy mieć ich mniej. 

Pamiętaj, żeby wyłączyć lub ograniczyć trening części modelu z wytrenowanymi już wagami (trainable=False). 

In [3]:
model1 = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=train_ds[0].shape,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model1.trainable = False

258076736/258076736 [==============================] - 2s 0us/step


### Zadanie 1: Wytrenuj model na swoich danych ###
**(Zadanie na ocenę 3)**

Wytrenuj wybrany model na swoich danych. Omów eksperyment i wyniki (100 słów). 

In [4]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    model1,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

print("/n/n")
model.evaluate(train_ds, train_labels)
model.evaluate(test_ds, test_labels)

Epoch 1/50
65/65 [==============================] - 48s 338ms/step - loss: 2.2484 - accuracy: 0.4394 - val_loss: 1.0126 - val_accuracy: 0.6206
Epoch 2/50
65/65 [==============================] - 16s 244ms/step - loss: 0.9161 - accuracy: 0.6540 - val_loss: 1.0764 - val_accuracy: 0.6342
Epoch 3/50
65/65 [==============================] - 18s 280ms/step - loss: 0.7739 - accuracy: 0.7182 - val_loss: 0.9465 - val_accuracy: 0.6634
Epoch 4/50
65/65 [==============================] - 18s 282ms/step - loss: 0.6444 - accuracy: 0.7630 - val_loss: 0.8088 - val_accuracy: 0.7276
Epoch 5/50
65/65 [==============================] - 16s 249ms/step - loss: 0.5637 - accuracy: 0.7903 - val_loss: 0.9699 - val_accuracy: 0.6751
Epoch 6/50
65/65 [==============================] - 16s 249ms/step - loss: 0.7044 - accuracy: 0.7358 - val_loss: 1.0509 - val_accuracy: 0.6732
Epoch 7/50
65/65 [==============================] - 16s 245ms/step - loss: 0.4600 - accuracy: 0.8282 - val_loss: 0.9870 - val_accuracy: 0.6732

[0.4657512307167053, 0.8328791856765747]

Jako zbiór wybrałem tensorflow Kwaity a jako model EfficientNetB7. Przy parametrach z tutoriala nosiagnął on całkiem dobre wyniki, ponieważ evaluate na zbiorze testowym wyniosło ok. 0,8. 
Model został wgrany jako Model1 z parametrami jak w poleceniu: wagi z imagnetu, bez ostatniej warstwy, na 1000 klas oraz ograniczeniem treningu. Następnie wytrenowany z trzema wartstwami, dwie relu i jedna softmax. Został również dodany early stopping ze względu na to, że model jest już wytrenowany więc nie powinnien potrzebować zbyt dużo iteracji. 
Wyniki accuracy oraz loss nie są idealne, myślę że wynika to z trudności zbioru. Łatwiej byłoby na pewno odróżniać przedmiot or roślini a nie gatunki roślin, niemniej z pewnością wyuczony model przy tych wynikach poradził by sobie lepiej niż przeciętny człowiek nieznający nawet nazw kwiatów. 

### Zadanie 2: Dodatkowe sieci ###
**(Zadanie na ocenę 4, po wykonaniu  zadania 1)**

Przeprowadź to samo na dwóch dodatkowych sieciach i omów wyniki (100 słów). 

Czyli jeśli w zadaniu 1 użyliśmy np. VGG to teraz wybieramy sobie np. ResNet i MobileNet. 

In [17]:
model2 = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=train_ds[0].shape,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model2.trainable = False

model3 = tf.keras.applications.ResNet152V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=train_ds[0].shape,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
model3.trainable = False

In [11]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(60, activation='relu')
dense_layer_2 = layers.Dense(30, activation='relu')
dense_layer_3 = layers.Dense(10, activation='relu')
prediction_layer = layers.Dense(5, activation='softplus')


model = models.Sequential([
    model2,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=10,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

print("/n/n")
model.evaluate(train_ds, train_labels)
model.evaluate(test_ds, test_labels)

Epoch 1/50
65/65 [==============================] - 8s 80ms/step - loss: 4.5797 - accuracy: 0.2399 - val_loss: 4.0378 - val_accuracy: 0.2549
Epoch 2/50
65/65 [==============================] - 4s 65ms/step - loss: 4.4970 - accuracy: 0.2579 - val_loss: 4.0230 - val_accuracy: 0.2665
Epoch 3/50
65/65 [==============================] - 4s 66ms/step - loss: 4.4777 - accuracy: 0.2856 - val_loss: 4.0078 - val_accuracy: 0.2918
Epoch 4/50
65/65 [==============================] - 5s 73ms/step - loss: 4.4493 - accuracy: 0.3294 - val_loss: 4.0359 - val_accuracy: 0.3307
Epoch 5/50
65/65 [==============================] - 4s 65ms/step - loss: 2.7094 - accuracy: 0.2910 - val_loss: 1.5767 - val_accuracy: 0.1848
Epoch 6/50
65/65 [==============================] - 4s 65ms/step - loss: 1.5289 - accuracy: 0.2706 - val_loss: 1.9175 - val_accuracy: 0.1829
Epoch 7/50
65/65 [==============================] - 4s 65ms/step - loss: 1.5679 - accuracy: 0.3290 - val_loss: 1.5739 - val_accuracy: 0.3191
Epoch 8/50
65

[1.207165241241455, 0.5340599417686462]

In [18]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(60, activation='relu')
dense_layer_2 = layers.Dense(40, activation='relu')
dense_layer_3 = layers.Dense(20, activation='relu')
dense_layer_4 = layers.Dense(10, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')


model = models.Sequential([
    model3,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    prediction_layer
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

print("/n/n")
model.evaluate(train_ds, train_labels)
model.evaluate(test_ds, test_labels)

Epoch 1/50
65/65 [==============================] - 22s 207ms/step - loss: 145.2508 - accuracy: 0.2589 - val_loss: 111.3391 - val_accuracy: 0.2471
Epoch 2/50
65/65 [==============================] - 11s 169ms/step - loss: 44.7720 - accuracy: 0.2818 - val_loss: 20.3724 - val_accuracy: 0.2918
Epoch 3/50
65/65 [==============================] - 11s 173ms/step - loss: 20.1206 - accuracy: 0.3319 - val_loss: 33.7761 - val_accuracy: 0.2451
Epoch 4/50
65/65 [==============================] - 11s 170ms/step - loss: 22.6072 - accuracy: 0.3630 - val_loss: 16.8909 - val_accuracy: 0.3872
Epoch 5/50
65/65 [==============================] - 11s 164ms/step - loss: 13.2274 - accuracy: 0.4034 - val_loss: 16.2878 - val_accuracy: 0.3677
Epoch 6/50
65/65 [==============================] - 11s 171ms/step - loss: 15.2012 - accuracy: 0.3839 - val_loss: 21.0091 - val_accuracy: 0.3113
Epoch 7/50
65/65 [==============================] - 11s 163ms/step - loss: 10.1798 - accuracy: 0.4292 - val_loss: 19.2193 - val_

[13.958308219909668, 0.42415985465049744]

V1 stara wersja

---


W kolejnych eksperymentach zdecydowałem się już na inne modele - ResNet50V2 oraz ResNet152V2 w celu porównania dwóch modeli z tej samej grupy. W teori 152V2 powinien sobie poradzić nieco lepiej niż 50V2 co chciałbym zaobserwować. Zmieniłem też parametry uczenia oraz dodałem jedną wartwę. Oczywiście modele 2 i 3 będą porównywane z wcześniejszym model1 EfficientNetB7.
Po treniowaniu na kwiatach wyniki modeli 2 i 3 okazały się bardzo słabe (accuracy 0.32 oraz 0.26) oraz dodatkowo odwrotne od moich oczekiwań. Model 50V2 powinien być gorszy niż 152V2 a poradził sobie sporo lepiej w accuracy bo aż o 0.06 punktu lepiej. Nie zmienia to faktu że moje stwierdzenie z zadania 1, że model poradzi sobie lepiej z rozpoznawaniem kwiatów od człowieka przestaje być prawdziwe dla tych modeli. Model 2 i 3 w przypadku kwiatów się nie sprawdziły, możliwe też że zmiany parametów oraz dodanie warsty mogły w tym przeszkodzić.

V2 nowa wersja po poprawie


---



W kolejnych eksperymentach zdecydowałem się już na inne modele - ResNet50V2 oraz ResNet152V2 w celu porównania dwóch modeli z tej samej grupy. W teori 152V2 powinien sobie poradzić nieco lepiej niż 50V2 co chciałbym zaobserwować. Zmieniłem też parametry uczenia oraz dodałem jedną wartwę. Oczywiście modele 2 i 3 będą porównywane z wcześniejszym modelem EfficientNetB7. W porównaniu z wersją pierwszą z przed poprawy, gdzie accuracy wynosiło 0.20-0.30 zmieniłem warstwy oraz funkcje aktywacyjną ostatniej warstwy na softplus. 


---


Po treniowaniu na kwiatach wyniki modeli 2 i 3 okazały się nieco słabsze (0.53 oraz 0.42) niż EfficientNetB7, ale lepsze niż poprzednie wyniki bliskie losowym wyborom. Model 50V2 powinien być gorszy niż 152V2 a poradził sobie sporo lepiej w accuracy bo aż o 0.09 punktu lepiej. W tym przypadku stwierdzenie z zadania 1 nadal ma sens. Zwykły człowiek nieznający się na kwiatach poradzi sobie gorzej, ale w odróżeniu od modelu z zadania 1 średnio obeznana z roślinami osoba może modele z zadania 2 pokonać.

### Zadanie 3: Trening od zera i porównanie ###
**(Zadanie na ocenę 5, po wykonaniu zadania 1 i 2)**

Spróbuj skonstruować swój własny model i wytrenować go 'od zera' na tych samych danych. Porównaj i omów swój ekeperyment i wyniki (100 słów).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
from keras.utils.np_utils import to_categorical

np_config.enable_numpy_behavior()

(X_train, y_train), (X_test, y_test) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"],
    batch_size=-1,
    as_supervised=True,
)



X_train = tf.image.resize(X_train, (32, 32))
X_test = tf.image.resize(X_test, (32, 32))

#y_train = to_categorical(y_train, num_classes=10)
#y_test = to_categorical(y_test, num_classes=10)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print("min: {}, max:{}".format(np.min(X_train), np.max(X_train)))
X_train = X_train.astype('float32') / np.max(X_train)
X_test = X_test.astype('float32') / np.max(X_test)
print("min: {}, max:{}".format(np.min(X_train), np.max(X_train)))


Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

print(y_train[0])
print(Y_train[0])

(2569, 32, 32, 3)
(1101, 32, 32, 3)
(2569,)
(1101,)
min: 0.0, max:255.0
min: 0.0, max:1.0
tf.Tensor(2, shape=(), dtype=int64)
[0. 0. 1. 0. 0.]


In [ ]:
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import  BatchNormalization
from keras.layers import RandomFlip
from keras.layers import RandomZoom
from tensorflow.keras.layers import Dropout
from keras.layers import Dense, Flatten

model_mega = Sequential([
    RandomFlip("horizontal", input_shape=(32, 32, 3)),
    RandomZoom(0.2, 0.2),

    Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(32, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),


    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),


    Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    #konczymy filtrowanie, feature extraction itp i zaczyna sie normalny, klasyczny klasyfikator
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')
], name="model_mega")

model_mega.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

es = EarlyStopping(patience=10, monitor="val_loss")

In [ ]:
model_mega.fit(X_train, Y_train, epochs=100, batch_size=256, validation_split=0.1, 
               #callbacks=[es]
               )

model_mega.evaluate(X_train, Y_train)
model_mega.evaluate(X_test, Y_test)

Epoch 1/100
10/10 [==============================] - 2s 151ms/step - loss: 1.2300 - accuracy: 0.4905 - val_loss: 3.7443 - val_accuracy: 0.2296
Epoch 2/100
10/10 [==============================] - 2s 218ms/step - loss: 1.1555 - accuracy: 0.5381 - val_loss: 4.3622 - val_accuracy: 0.2296
Epoch 3/100
10/10 [==============================] - 1s 141ms/step - loss: 1.1182 - accuracy: 0.5346 - val_loss: 4.0952 - val_accuracy: 0.2296
Epoch 4/100
10/10 [==============================] - 1s 136ms/step - loss: 1.1228 - accuracy: 0.5324 - val_loss: 4.8759 - val_accuracy: 0.2296
Epoch 5/100
10/10 [==============================] - 1s 139ms/step - loss: 1.1121 - accuracy: 0.5528 - val_loss: 4.7464 - val_accuracy: 0.2296
Epoch 6/100
10/10 [==============================] - 1s 137ms/step - loss: 1.1159 - accuracy: 0.5484 - val_loss: 4.2432 - val_accuracy: 0.2296
Epoch 7/100
10/10 [==============================] - 1s 145ms/step - loss: 1.1875 - accuracy: 0.5104 - val_loss: 4.3974 - val_accuracy: 0.2296

[0.9204573035240173, 0.6512261629104614]

Jako mój model skopiowałem z drobnymi modyfikacjami model z zajęć "mega", który był połączeniem wszystkich dotychczasowych narzędzi - BatchNormalization, RandomZoom i RandomFlip, Dropout, MaxPooling, Flatten. Zrezygnowałem tylko z EarlyStoppingu ze względu na to że na początku uczenia model radził sobie wyjątkowo stabilnie źle przez co poddawałby się za wcześnie. 
Po tej początkowej fazie kilku iteracji niepowidzeń model poradził sobie całkiem nieźle bo osiągnał na zbiorze treningowym accuracy 0.65 a miejscami podczas uczenia nawet ponad 0.70. Jest to zdecydowanie lepiej niż preuczone modele 2 i 3 z zadania 2. Nie pobił on jednak modelu 1 z zadania 1 który miał wynik w okolicy 0.79. 
Myślę, że przy doborze odpowiednich technik i liczb wyniki można by jeszcze trochę podciągnąć tak, aby dorównywał modelowi 1. Z drugiej strony model 1 jest już gotowy i dostosowanie go jest szybsze niż uczenie nowego modelu, którym dążymy jedynie do wyrównania a nie do znaczego polepszenia wyników. 

To wszystko, dziękuję. Wypełniony notatnik zapisz jako .ipynb i oddaj w Teams. 